Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy 
from scipy import stats
from scipy.stats import shapiro
import statsmodels as sm
from statsmodels.stats.power import TTestIndPower

In [9]:
df = pd.read_csv ('result1.csv')
df.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [11]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [12]:
df.version.unique()

array(['gate_30', 'gate_40'], dtype=object)

In [14]:
df.nlargest(10, ['sum_gamerounds'])

,userid,version,sum_gamerounds,retention_1,retention_7
57702,6390605,gate_30,49854,0,1
7912,871500,gate_30,2961,1,1
29417,3271615,gate_40,2640,1,0
43671,4832608,gate_30,2438,1,1
48188,5346171,gate_40,2294,1,1
46344,5133952,gate_30,2251,1,1
87007,9640085,gate_30,2156,1,1
36933,4090246,gate_40,2124,1,1
88328,9791599,gate_40,2063,1,1
6536,725080,gate_40,2015,1,1


In [15]:
# Удаляем выброс
df_new = df[df.sum_gamerounds < 3000]

In [17]:
df_new.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018800e+04,90188.000000,90188.000000,90188.000000
mean,4.998397e+06,51.320253,0.445214,0.186056
std,2.883298e+06,102.682719,0.496992,0.389154
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512211e+06,5.000000,0.000000,0.000000
50%,4.995804e+06,16.000000,0.000000,0.000000
75%,7.496461e+06,51.000000,1.000000,0.000000
max,9.999861e+06,2961.000000,1.000000,1.000000


In [24]:
df_new.userid.nunique()

90188

In [18]:
# Определим размеры выборок.

A = df_new.version.groupby(df.version == 'gate_30').count()
B = df_new.version.groupby(df.version == 'gate_40').count()
print(A)
print(B)
     

version
False    45489
True     44699
Name: version, dtype: int64
version
False    44699
True     45489
Name: version, dtype: int64


Проверка на нормальность распределения по метрике retention_1

In [22]:
alpha = 0.05
st = shapiro(df_new.retention_1)
st

C:\Users\admin\anaconda3\lib\site-packages\scipy\stats\_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.6318845152854919, pvalue=0.0)

Так как pvalue<alpha распределение не является нормальным

Проверка на нормальность распределения по метрике retention_7

In [25]:
alpha = 0.05
st = shapiro(df_new.retention_7)
st

ShapiroResult(statistic=0.47385239601135254, pvalue=0.0)

Так как pvalue<alpha распределение не является нормальным

Используем критерий Манна -Уитни

In [29]:
control = df_new.query('version == "gate_30"')

test = df_new.query('version == "gate_40"')

In [30]:
mw_stats = scipy.stats.mannwhitneyu(x=control.retention_1.values, y = test.retention_1.values)

mw_stats

MannwhitneyuResult(statistic=1022670128.0, pvalue=0.073922385401239)

Так как pvalue>alpha стат значимых различий между группами не обнаружено

In [31]:
mw_stats = scipy.stats.mannwhitneyu(x=control.retention_7.values, y = test.retention_7.values)

mw_stats

MannwhitneyuResult(statistic=1024975889.5, pvalue=0.0015918697815176374)

Так как pvalue<alpha обнаружены cтат значимые различий между группами

In [35]:
control.retention_7.value_counts().sort_values()

1     8501
0    36198
Name: retention_7, dtype: int64

In [36]:
test.retention_7.value_counts().sort_values()

1     8279
0    37210
Name: retention_7, dtype: int64

Таким образом по метрике retention_7 лучше себя показывает группа А


Расчёт мощности теста

In [37]:
C_mean = control.retention_7.values.mean()
T_mean = test.retention_7.values.mean()

C_std = control.retention_7.values.std()
T_std = test.retention_7.values.std()

print(len(control.retention_7.values), len(test.retention_1.values))

44699 45489


In [38]:
n = len(control.retention_7.values)
     
S = np.sqrt((T_std**2 +C_std**2)/2)
     
ef = float((C_mean-T_mean)/S)

In [44]:
alpha = 0.05
analysis = TTestIndPower()
analysis = sm.stats.power.TTestIndPower()
result = analysis.solve_power(ef, power=None,
                              nobs1=n, ratio=1.0, alpha=alpha)

result

0.8817172245738318

При анализе результатов теста по метрике retention_7 выявлены стат значимые различия между группами и лучше себя показала группа А, мощность теста составила 88%. Таким образом не рекомендуется применять результаты эксперимента.